In [1]:
from scraping_player_data import *

In [33]:
scrape = ScrapePlayerData()
links = scrape.collect_player_links()
df_list = []
names_list = []


for link in links[0:100]:
    df = scrape.scrape_statistic_tables(link)
    name = scrape.scrape_player_name(link)
    names_list.append(name)
    df_list.append(df)
    time.sleep(.5)
df_list

c:\Users\bengu\Documents\mlb_player_projection\web_scraping\scraping_player_data.py:47: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 47 of the file c:\Users\bengu\Documents\mlb_player_projection\web_scraping\scraping_player_data.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  new_soup = BeautifulSoup(str(comments))


[    level_0  Year Age_x Tm_x Lg_x  G_x PA_x   AB    R    H  ...  RAR   WAR  \
 0         0  1963    22  CIN   NL  157  696  623  101  170  ...   23   2.4   
 1         1  1964    23  CIN   NL  136  558  516   64  139  ...   12   1.3   
 2         2  1965    24  CIN   NL  162  757  670  117  209  ...   53   5.6   
 3         3  1966    25  CIN   NL  156  701  654   97  205  ...   38   4.1   
 4         4  1967    26  CIN   NL  148  650  585   86  176  ...   43   4.8   
 5         5  1968    27  CIN   NL  149  692  626   94  210  ...   46   5.5   
 6         6  1969    28  CIN   NL  156  731  627  120  218  ...   61   6.6   
 7         7  1970    29  CIN   NL  159  730  649  120  205  ...   46   4.8   
 8         8  1971    30  CIN   NL  160  709  632   86  192  ...   46   5.1   
 9         9  1972    31  CIN   NL  154  731  645  107  198  ...   55   6.1   
 10       10  1973    32  CIN   NL  160  752  680  115  230  ...   80   8.3   
 11       11  1974    33  CIN   NL  163  771  652  1

Continuous Categories

In [34]:
categories = ['Age_x', 'G_x', 'PA_x', 'AB', 'R','H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'BA', 'OBP','SLG', 'OPS', 'OPS+', 'TB', 'rOBA', 'Rbat+','BAbip', 'ISO', 'HR%', 'SO%', 'BB%', 'WPA', 'cWPA', 'RE24', 'RS%','SB%', 'XBT%', 'Rbat','Rbaser', 'Rdp', 'Rfield', 'Rpos', 'RAA', 'WAA', 'Rrep', 'RAR', 'WAR','waaWL%', '162WL%', 'oWAR', 'dWAR', 'oRAR']

In [35]:
pct_cats = ["HR%", "SO%", "BB%", "RS%", "SB%", "XBT%", "cWPA"]

In [36]:
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np

In [37]:
def clean_df(df, pct_cats):
    """
    param: df(pandas dataframe)
    param: pct_cats(list)
    This function returns a dataframe with percentage values cleaned if you correctly supply the percentage categories
    """
    for cat in pct_cats:
        try:
            for i in df[cat].index:
                try:
                    df[cat][i] = float(df[cat][i][:-1])
                except:
                    df[cat][i] = df[cat][i]
        except:
            pass
    return df


In [38]:
def flatten_df(df, categories):
    """
    param: df(pandas dataframe)
    param: categories(list)
    Will return a flattened DataFrame of length 1 with your desired categories. Removes categories that will not be found in older data.
    """
    try:
        df = df[categories]
    except:
        new_cats = categories.copy()
        new_cats.remove('WPA')
        new_cats.remove('cWPA')
        new_cats.remove('RE24')
        new_cats.remove('XBT%')
        df = df[new_cats]
    s = df.stack()
    s.index = [f"{x}_{y}" for x,y in s.index]
    s = s.to_frame('').T
    return s

In [39]:
edge = True
for df in df_list:
    if edge:
        df = clean_df(df, pct_cats)
        curr_df = flatten_df(df, categories)
        new_df = curr_df
        edge = False
    else:
        df = clean_df(df, pct_cats)
        curr_df = flatten_df(df, categories)
        new_df = new_df.append(curr_df)

c:\Users\bengu\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
c:\Users\bengu\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [131]:
scaler = StandardScaler()
columns = new_df.columns
scaled_data = scaler.fit_transform(new_df)
new_df = pd.DataFrame(
    scaled_data,
    columns=columns
)
new_df

,0_Age_x,0_G_x,0_PA_x,0_AB,0_R,0_H,0_2B,0_3B,0_HR,0_RBI,...,25_RAA,25_WAA,25_Rrep,25_RAR,25_WAR,25_waaWL%,25_162WL%,25_oWAR,25_dWAR,25_oRAR
0,0.967116,1.280685,1.420760,1.415575,1.263444,1.073065,0.524355,0.896669,-0.395013,-0.234683,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.184213,0.579214,0.563610,0.624266,0.046417,0.434336,0.694049,0.227513,0.276126,0.665471,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.391452,1.100306,1.177824,1.211366,0.414355,0.827400,1.033438,0.227513,0.084372,1.019103,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.184213,0.418878,0.554443,0.466004,0.499264,0.352448,1.033438,-0.218591,1.234895,0.858361,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-1.335541,-1.044189,-1.013178,-0.994088,-1.057398,-1.121543,-1.087740,-1.110799,-0.874398,-1.070540,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.391452,1.360853,1.329086,1.247103,1.574776,1.466129,2.390991,-0.218591,2.577173,2.626521,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.391452,1.340811,1.283249,1.354312,0.697385,1.122198,0.863743,-0.664695,1.618403,1.276290,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.391452,0.599256,0.788211,0.716160,1.121929,0.811022,0.694049,0.896669,-0.778521,-0.106090,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,-0.759877,-0.583223,-0.431050,-0.509092,-0.208310,-0.417303,0.184967,-0.664695,0.276126,0.247542,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,-1.335541,0.278584,-0.101024,-0.008781,-0.236613,-0.302659,-0.408963,0.004461,-0.682644,-0.716908,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
no_na_df = new_df.dropna(axis=1)

In [133]:
model = KMeans(n_clusters=5)
model.fit(no_na_df)

KMeans(n_clusters=5)

In [134]:
predictions = model.predict(no_na_df)

In [135]:
name_pred = {}
for i in range(len(names_list)):
    name = names_list[i]
    pred = predictions[i]
    name_pred[name] = pred
index = [0]
predictions_df = pd.DataFrame(
    columns=["name", "group", "HR", "WAR", "G", "OPS"]
)

In [136]:
import hvplot.pandas
# predictions_df = predictions_df.swapaxes(0, 1)
predictions_df["name"] = names_list
predictions_df["group"] = predictions

In [137]:
wars = []
hrs = []
ops = []
gs = []
for i in range(len(df_list)):
    hrs.append(df_list[i]["HR"].astype("int").sum())
    wars.append(df_list[i]["WAR"].astype("float").sum())
    gs.append(df_list[i]["G_x"].astype("int").sum())
    ops.append(df_list[i]["OPS"].astype("float").mean())
predictions_df["HR"] = hrs
predictions_df["WAR"] = wars
predictions_df["G"] = gs
predictions_df["OPS"] = ops


In [1]:
predictions_df[predictions_df["group"] == 0]

NameError: name 'predictions_df' is not defined

In [130]:
predictions_df.hvplot.points(x="HR", y="group")

:Points   [HR,group]

In [125]:
war_group = predictions_df[["HR", "group"]]
war_group.corr()

,HR,group
HR,1.000000,0.568245
group,0.568245,1.000000
